# Demographics
Merge multiple reports to combine resident information to update database

In [ ]:
import pandas as pd

# 1. Importing All Active Residents

In [ ]:
# Importing Active Residents
ActiveResidents = pd.read_excel("All_Resident_Demographics/allactiveresidents.xlsx")
ActiveResidents.shape

In [ ]:
#ActiveResidents.head()

In [ ]:
ActiveResidents['In All Active'] = 'In All Active'

In [ ]:
#Capitalizing Names
ActiveResidents['Household Member Name'] = ActiveResidents['Household Member Name'].str.upper() 

In [ ]:
#Drop SSN Column if Present
ActiveResidents.drop('SSN',axis=1, inplace = True)
ActiveResidents.shape

In [ ]:
#Locating Duplicate Residents
DupResidents = ActiveResidents[ActiveResidents.duplicated(subset=['Community','Household Member Name','Apt#','Relation', 'DOB'],keep=False)]
DupResidents.shape

In [ ]:
#print(DupResidents)

In [ ]:
#Checking Values
ActiveResidents['Community'].value_counts()

# 2. Importing Master List

In [ ]:
#Importing Master List
MasterList = pd.read_excel("Masterlist 11.13.2020.xlsx")
MasterList.shape

In [ ]:
MasterList['In MasterList'] = 'In MasterList'

In [ ]:
#Capitalizing Names
MasterList['Residents/Non-Residents Name'] = MasterList['Residents/Non-Residents Name'].str.upper() 

In [ ]:
#MasterList.columns

# 3. Merge All Active and Master List

In [ ]:
#Find matches on Community, Name and DOB
MovedOutResidents = pd.merge(ActiveResidents, MasterList, 
                                left_on= ('Community','Household Member Name','DOB'), 
                            right_on = ('Community','Residents/Non-Residents Name','DOB'), 
                            how='right', suffixes=['_AllActive', '_ML'])
MovedOutResidents.shape

In [ ]:
#Find matches on Community, Name and DOB: MovedIn
MovedIn = pd.merge(ActiveResidents, MasterList, 
                                left_on= ('Community','Household Member Name', 'DOB','Relation'), 
                            right_on = ('Community','Residents/Non-Residents Name', 'DOB', 'Relation_Master'), 
                            how='left', suffixes=['_AllActive', '_ML'])
MovedIn.shape

In [ ]:
#Locating Duplicate values
#DupMovedIn = MovedIn[MovedIn.duplicated(subset=['Community','Household Member Name', 'DOB','Relation'],keep=False)]
#DupMovedIn.shape

In [ ]:
#DupMovedIn.head()

# 4. Export Moved In and Moved Out Residents to Concatenate in Excel

In [ ]:
#MovedIn.to_csv('MovedIn_11.13.2020.csv')

In [ ]:
#MovedOutResidents.to_csv('MovedOutResidents_11.13.2020.csv')

# 5. Import Resident Demographics Report: Demographics

In [ ]:
# Importing Demographics
Demographics = pd.read_excel("householddemographics.xlsx")
Demographics.shape

# 6. Fixing Race Columns
Replace "X" with race using a dictionary then collapsing multiple columns into one column

In [ ]:
#Mapping numbers 
#Replacing x values to true/false for analysis

# Create the dictionary: true_false
Hawaiian_Pacific = { 'X':'Hawaiian/ Pacific Islander'}
Hispanic_Latino = { 'X':'Hispanic'}
Am_Indian_Alaskan = { 'X':'Am Indian Alaskan'}
Asian = { 'X':'Asian'}
Black_African = { 'X':'Black African'}
White = { 'X':'White'}
None_0 = { 'X':'None'}
Other = { 'X':'Other'}
Multi_Race = { 'X':'Multi Race'}

# Map columns to the new columns with true_false
Demographics['Hawaiian Pacific'] = Demographics['Hawaiian Pacific'].map(Hawaiian_Pacific)
Demographics['Hispanic Latino'] = Demographics['Hispanic Latino'].map(Hispanic_Latino)
Demographics['AmIndian Alaskan'] = Demographics['AmIndian Alaskan'].map(Am_Indian_Alaskan)
Demographics['Asian'] = Demographics['Asian'].map(Asian)
Demographics['Black African'] = Demographics['Black African'].map(Black_African)
Demographics['White'] = Demographics['White'].map(White)
Demographics['None'] = Demographics['None'].map(None_0)
Demographics['Other'] = Demographics['Other'].map(Other)
Demographics['Multi Race'] = Demographics['Multi Race'].map(Multi_Race)

In [ ]:
Demographics.columns

In [ ]:
#Collapsing Race/Ethnicity columns into one column: 'Race_Ethnicity'
source_col_loc = Demographics.columns.get_loc('Hispanic Latino') # column position starts from 0

Demographics['Race'] = Demographics.iloc[:,source_col_loc+1:source_col_loc+9].apply(lambda x: ",".join(x.dropna()), axis=1)
Demographics.shape

In [ ]:
Demographics['Race'].value_counts()

# 7. Cleaning up strings
Replace All Multi-Race Residents with "Multi Race"

In [ ]:
Demographics['Race'] = Demographics['Race'].str.replace('Black/ African,Other,Multi-Race', 'Multi Race')
Demographics['Race'] = Demographics['Race'].str.replace('Asian,Other,Multi-Race', 'Multi Race')
Demographics['Race'] = Demographics['Race'].str.replace('White,Other,Multi-Race', 'Multi Race')
Demographics['Race'] = Demographics['Race'].str.replace('Asian,Black/ African,Other,Multi-Race', 'Multi Race')

In [ ]:
Demographics['Race'].value_counts()

In [ ]:
# Capitalize Names for Merge
Demographics['Member Name'] = Demographics['Member Name'].str.upper()

## Replace Strings for Gender

In [ ]:
Demographics['Gender'] = Demographics['Gender'].str.replace('M', 'Male')
Demographics['Gender'] = Demographics['Gender'].str.replace('F', 'Female')

In [ ]:
#Locating Duplicate values
DupDemographics = Demographics[Demographics.duplicated(subset=['Member Name', 'Apt #','Relation', 'Current Age'],keep=False)]
DupDemographics.shape

# 8. Left Merge Demographics with Moved In Residents
Merge demographics to current database Masterlist

In [ ]:
# Merging Demographics and MoveIn_Residents
Masterlist_Demographics = pd.merge(Moved_In_Residents, Demographics, how='left', 
                                   left_on=['Community','Household Member Name', 'Apt#'], 
                               right_on=['Community','Member Name', 'Apt #'], 
                                   suffixes=['_Master', '_Demo'])
Masterlist_Demographics.shape

In [ ]:
#Export and remove columns
#Masterlist_Demographics.to_csv('Masterlist_Demographics_11.10.2020.csv')